In [2]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline
from fish.ephys.ephys import load, estimate_onset, chop_trials

In [4]:
from triggered_average import trigger_data, match_cam_time, load_single_plane

In [5]:
from fish.util.fileio import load_image

In [ ]:
dirs = {}
dirs['ephys'] = r'R:/davis/data/ephys/20171004/7dpf_cy171xec43_f1_omr_opto_1.10chFlt'
dirs['ims'] = r'F:/davis/20171004/7dpf_cy171xec43_f1_omr_opto_1_20171004_113405/'
from fish.image.vol import get_stack_dims

dims = get_stack_dims(dirs['ims'])[::-1]
single_plane = False
if dims[0] == 50:
    single_plane = True

fnames = sorted(glob(dirs['ims'] + 'TM*'))
# Number of timepoints per file is 50 for high speed single plane
num_frames = len(fnames)
if single_plane:
    num_frames = len(fnames) * 50
print(len(fnames))
print(num_frames)

In [ ]:
ep_dat = load(dirs['ephys'])
cam_times = estimate_onset(ep_dat[2], threshold=3, duration=50)[:num_frames]
laser_on = estimate_onset(ep_dat[3], threshold=3.7, duration=400)
laser_off = len(ep_dat[3]) - estimate_onset(ep_dat[3][::-1], threshold=3.7, duration=400)
print(len(cam_times))

In [ ]:
trials = chop_trials(ep_dat[4])
# for each trial onset look for the nearest cam time before the onset.
# for each offset, look for the nearest cam time after offset
trials_cam = {}

for key, val in trials.items():
    pre = match_cam_time(val[0], cam_times, timing='pre')
    post = match_cam_time(val[1], cam_times, timing='post')
    trials_cam[key] = [pre, post]

In [ ]:
#todo: use multiprocessing

In [ ]:
# Get all events
window=np.arange(-50, 110)
triggered = {}
for key, value in trials_cam.items():
    if key > 0:
        print(key)
        
        # don't trigger on values where we can't get a full window
        trigs = list(filter(lambda v: v + window[-1] < num_frames, value[-1]))
        print(trigs)
        triggered[key] = trigger_data(trigs, window, fnames, average=True, z=None)

In [ ]:
import pyqtgraph as pq
%gui qt

In [ ]:
pq.image(triggered[1.0], title='roi 1.0')
pq.image(triggered[2.0], title='roi 2.0')
pq.image(triggered[3.0], title='roi 3.0')

In [ ]:
pq.image(triggered[1.0][:,-15])